<a href="https://colab.research.google.com/github/Noman-ashraf1/AutoEssayGrader/blob/main/Essay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install datasets

#Libraries import

In [11]:
pip install nltk


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.utils import to_categorical
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from imblearn.over_sampling import RandomOverSampler
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#Load the dataset

In [13]:
#df = pd.read_csv('/content/essay_llama3_8B_groq.csv')

df = pd.read_csv("/content/essay_llama3_8B_groq.csv", quotechar='"', engine="python")



# Take a random sample of 10000 rows
#df = df.sample(n=10000, random_state=42).reset_index(drop=True)
df.drop(columns=['Unnamed: 0', 'essay_id', 'cleaned_text'], axis=1, inplace=True)
df.head()

,full_text,score
0,Many people have car where they live. The thin...,3
1,I am a scientist at NASA that is discussing th...,3
2,People always wish they had the same technolog...,4
3,"We all heard about Venus, the planet without a...",4
4,"Dear, State Senator\n\nThis is a letter to arg...",3


In [14]:
df.shape

(17307, 2)

#Data prepeocessing

In [15]:
stemer = PorterStemmer()
nltk.download('stopwords')
stop_word = set(stopwords.words('english'))
def cleaned_text(text):
  text = text.lower()
  text =  re.sub(r'[^A-Za-z\s]', '', text)
  text = re.sub(r'[^\w\s]', '', text)

  words = text.split()
  stemmed = [stemer.stem(word) for word in words if word.lower not in stop_word]



  return ' '.join(stemmed)




df['clean_text'] = df['full_text'].apply(cleaned_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,full_text,score,clean_text
0,Many people have car where they live. The thin...,3,mani peopl have car where they live the thing ...
1,I am a scientist at NASA that is discussing th...,3,i am a scientist at nasa that is discuss the f...
2,People always wish they had the same technolog...,4,peopl alway wish they had the same technolog t...
3,"We all heard about Venus, the planet without a...",4,we all heard about venu the planet without alm...
4,"Dear, State Senator\n\nThis is a letter to arg...",3,dear state senat thi is a letter to argu in fa...


In [16]:
df['score'] = df['score'].astype('category').cat.codes

#Train_test_split

In [17]:
new_df = df.drop(columns=['full_text'],axis=1)
new_df.head()

,score,clean_text
0,2,mani peopl have car where they live the thing ...
1,2,i am a scientist at nasa that is discuss the f...
2,3,peopl alway wish they had the same technolog t...
3,3,we all heard about venu the planet without alm...
4,2,dear state senat thi is a letter to argu in fa...


In [18]:
X = new_df.drop(columns=['score'])
y = new_df['score']
X.head()

,clean_text
0,mani peopl have car where they live the thing ...
1,i am a scientist at nasa that is discuss the f...
2,peopl alway wish they had the same technolog t...
3,we all heard about venu the planet without alm...
4,dear state senat thi is a letter to argu in fa...


Handel Imbalnce dataset

In [19]:
df['score'].value_counts()

,count
score,
2,6280
1,4723
3,3926
0,1252
4,970
5,156


In [20]:
from imblearn.over_sampling import RandomOverSampler
import warnings

# Ignore warnings from sklearn.base
warnings.simplefilter(action='ignore', category=FutureWarning)

# Apply Random Over-Sampling
ros = RandomOverSampler(random_state=42)
x_resample, y_resample = ros.fit_resample(X, y)
data = pd.concat([x_resample, y_resample], axis=1)
data.head()



,clean_text,score
0,mani peopl have car where they live the thing ...,2
1,i am a scientist at nasa that is discuss the f...,2
2,peopl alway wish they had the same technolog t...,3
3,we all heard about venu the planet without alm...,3
4,dear state senat thi is a letter to argu in fa...,2


In [21]:
data.shape


(37680, 2)

In [22]:
# Perform train-validation split
x_train, x_test, y_train, y_test = train_test_split(data['clean_text'], data['score'], test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((30144,), (7536,), (30144,), (7536,))

#Load pretrained model

In [23]:
from transformers import AutoTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base", return_tensors='tf')

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=6)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#Tokenization

In [24]:
import tensorflow
# Tokenize input data
x_train_token = tokenizer(list(x_train), truncation=True, padding=True, max_length=512, return_tensors='tf')
x_test_token = tokenizer(list(x_test), truncation=True, padding=True, max_length=512, return_tensors='tf')


In [25]:
x_test_token

{'input_ids': <tf.Tensor: shape=(7536, 512), dtype=int32, numpy=
array([[   0,  179,    5, ...,    1,    1,    1],
       [   0, 1990,    5, ...,    2,    1,    1],
       [   0,  179,  127, ..., 3572,  936,    2],
       ...,
       [   0, 2987,  257, ..., 1504,   15,    2],
       [   0,  627, 2730, ...,    1,    1,    1],
       [   0,  179,    5, ...,    1,    1,    1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(7536, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [26]:
from datasets import Dataset

# Assuming x_train_token, x_test_token, y_train, y_test are your tokenized data and labels.

train_data = Dataset.from_dict(
    {
        'input_ids': x_train_token['input_ids'],
        'attention_mask': x_train_token['attention_mask'],
        'label': y_train.tolist()
    }
)

val_data = Dataset.from_dict(
    {
        'input_ids': x_test_token['input_ids'],
        'attention_mask': x_test_token['attention_mask'],
        'label': y_test.tolist()
    }
)


#fine tune the model

In [27]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1000,
    evaluation_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    learning_rate=5e-5,
    metric_for_best_model="eval_loss",  # Early stopping metric
    greater_is_better=False,  # False for loss
    load_best_model_at_end=True
)



# Initialize Trainer
trainer = Trainer(
    model=model,  # Your Transformer model
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
1000,0.868700,0.582585
2000,0.485400,0.467259


TrainOutput(global_step=2826, training_loss=0.5764312852129224, metrics={'train_runtime': 9042.3998, 'train_samples_per_second': 10.001, 'train_steps_per_second': 0.313, 'total_flos': 2.3794513492967424e+16, 'train_loss': 0.5764312852129224, 'epoch': 3.0})

#Model evaluation

In [28]:
# Assuming `test_data` is your test dataset
results = trainer.evaluate(eval_dataset=val_data)
print(results)



{'eval_loss': 0.4672589898109436, 'eval_runtime': 220.0738, 'eval_samples_per_second': 34.243, 'eval_steps_per_second': 1.072, 'epoch': 3.0}


In [30]:
model.save_pretrained("./fine_tune_model")
tokenizer.save_pretrained("./fine_tune_model")

('./fine_tune_model/tokenizer_config.json',
 './fine_tune_model/special_tokens_map.json',
 './fine_tune_model/vocab.json',
 './fine_tune_model/merges.txt',
 './fine_tune_model/added_tokens.json')

#Prediction system

In [35]:
tokenizer = RobertaTokenizer.from_pretrained('/content/fine_tune_model')

model = RobertaForSequenceClassification.from_pretrained("/content/fine_tune_model")

In [45]:




# Define the detect function
def detect(text, model, tokenizer, class_names):
    # Clean the input text
    text = cleaned_text(text)

    # Tokenize the input text
    inputs = tokenizer(
        text,
        padding=True,       # Add padding
        truncation=True,    # Ensure truncation
        max_length=200,     # Set a reasonable max_length
        return_tensors='tf' # Return TensorFlow tensors
    )

    # Perform inference with the model
    outputs = model(**inputs)
    logits = outputs.logits

    # Get the index of the predicted label
    predicted_label_index = tf.argmax(logits, axis=1).numpy()[0]

    # Map the index to the actual label
    predicted_label = class_names[predicted_label_index]

    return predicted_label


# Define class names
class_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6']

# Example text
text = """
Imagine being a top scientist at and Viking 1 spacecraft spotted the shadowy likeness of a human face. A lot of people thought of alien life possibility, but others thought it to be just a natural landform. What would your thoughts be about the image? The face of Mars is a natural landform. There are similar landforms on Earth, high-pixel photos taken, and not enough reliable information to support the alien life theory. To begin, there are similar landforms on Earth to the one on Mars. As stated in the text, 'what the picture actually shows is the Martian equivalent of a butte or mesa — landforms common around the American West.' Also stated in the text is Garvin giving his opinion, 'it reminds me most of Middle Butte in the Snake River Plain of Idaho.' Another issue with the alien life theory is the fact that NASA has high-quality cameras. Quoting the text, 'each pixel in the 2001 image spans 1.56 meters, compared to 43 meters per pixel in the best 1976 Viking photo.' In the result of having high-quality pictures, you could see small details clearly. As stated in the article, Garvin said, 'so, if there were objects in this picture like airplanes on the ground or Egyptian-style pyramids or even small shacks, you could see what they were!' Concluding that statement, nothing other than the landform that resembled a face was seen on Mars. Finally, there wasn't enough evidence to support the claim of alien life on Mars. The first pictures were blurry...
"""

# Perform prediction
predicted_label = detect(text, model, tokenizer, class_names)
print(f"Predicted Label: {predicted_label}")


Predicted Label: class_3
